Data Loader Script

In [1]:
import pandas as pd
import os

def load_dataset(csv_path):
    df = pd.read_csv(csv_path, header=None, names=["audio_path", "label"])
    # Fix path formatting for Windows/Linux compatibility
    df["audio_path"] = df["audio_path"].apply(lambda x: os.path.normpath(x.replace("\\", "/")))
    return df

# Load train/test data
train_df = load_dataset("train_dataset.csv")
test_df = load_dataset("test_dataset.csv")

print("Train distribution:\n", train_df["label"].value_counts())
print("\nTest distribution:\n", test_df["label"].value_counts())

Train distribution:
 label
neutral    134
happy      134
sad        133
angry      127
fear        70
emotion      1
Name: count, dtype: int64

Test distribution:
 label
sad        42
neutral    42
happy      42
angry      40
fear       22
emotion     1
Name: count, dtype: int64


Dataset Class

In [ ]:
from transformers import Wav2Vec2Processor
import librosa
import numpy as np
import torch

class TamilEmotionDataset(torch.utils.data.Dataset):
    def __init__(self, df, processor, max_length=16000*3):
        self.df = df
        self.processor = processor
        self.max_length = max_length
        
        # Auto-create label map from actual data
        self.labels = sorted(df["label"].unique().tolist())
        self.label_map = {label: idx for idx, label in enumerate(self.labels)}
        self.inverse_map = {v: k for k, v in self.label_map.items()}
        
        # Verify expected labels
        expected = {'angry', 'fear', 'happy', 'neutral', 'sad'}
        assert set(self.labels) == expected, f"Unexpected labels: {set(self.labels)-expected}"

    def __getitem__(self, idx):
        audio_path = self.df.iloc[idx]["audio_path"]
        label = self.df.iloc[idx]["label"]  # Get raw label text
        
        # Validate label before mapping
        if label not in self.label_map:
            raise ValueError(f"Invalid label '{label}' in row {idx}")
            
        label_idx = self.label_map[label]
       
        
        # Load and preprocess audio
        speech, sr = librosa.load(audio_path, sr=16000)
        
        # Pad/truncate to 3 seconds
        if len(speech) > self.max_length:
            speech = speech[:self.max_length]
        else:
            padding = self.max_length - len(speech)
            speech = np.pad(speech, (0, padding))
            
        # Process with Wav2Vec2
        inputs = self.processor(
            speech,
            sampling_rate=16000,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
            truncation=True
        )
        
        return {
            "input_values": inputs["input_values"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.long)
        }

Training Script

In [3]:
from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2ForSequenceClassification,
    TrainingArguments, 
    Trainer
)
import torch
from sklearn.metrics import f1_score, accuracy_score

# Initialize components
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    "facebook/wav2vec2-base",
    num_labels=5
)

# Create datasets
train_dataset = TamilEmotionDataset(train_df, processor)
test_dataset = TamilEmotionDataset(test_df, processor)

# Training configuration
training_args = TrainingArguments(
    output_dir="./tamil_ser_results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

# Metrics with class weights
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

d:\GitHub\SER v.03\venv\Lib\site-packages\transformers\configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\GitHub\SER v.03\venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyError: 'emotion'

4. Inference Script

In [ ]:
def predict_emotion(audio_path, model, processor):
    # Preprocess
    speech, sr = librosa.load(audio_path, sr=16000)
    inputs = processor(
        speech,
        sampling_rate=16000,
        padding="max_length",
        max_length=16000*3,
        return_tensors="pt",
        truncation=True
    )
    
    # Predict
    with torch.no_grad():
        logits = model(inputs.input_values).logits
    
    # Convert to label
    label_idx = logits.argmax(-1).item()
    return test_dataset.inverse_map[label_idx]

# Example usage
test_audio = "dataset/neutral/03_01_neu.wav"  # Replace with your file
prediction = predict_emotion(test_audio, model, processor)
print(f"Predicted emotion: {prediction}")